In [29]:
# coding: utf-8
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [108]:
import h1st as h1
h1.init()

In [109]:
from h1st.tuner import HyperParameterTuner
from examples.RayTune.tensorflow_mlp_classifier import TensorflowMLPClassifier
from examples.RayTune.sklearn_rf_classifier import SklearnRFClassifier

In [110]:
tuner = HyperParameterTuner()

In [111]:
# parameters = [
#     {
#         'name': 'lr',
#         'type': 'float',
#         'min': 0.001,
#         'max': 0.1,
#         'choice': []
#     },
#     {
#         'name': 'n_layer',
#         'type': 'int',
#         'min': 1,
#         'max': 8,
#         'choice': []
#     },
#     {
#         'name': 'units',
#         'type': 'int',
#         'min': None,
#         'max': None,
#         'choice': [4, 8, 16, 32, 64]
#     }    
# ]

parameters = [
    {
        'name': 'n_estimators',
        'type': 'int',
        'min': None,
        'max': None,
        'choice': [50, 100, 150]
    },
    {
        'name': 'max_depth',
        'type': 'int',
        'min': 3,
        'max': 8,
        'choice': []
    },
    {
        'name': 'criterion',
        'type': 'str',
        'min': None,
        'max': None,
        'choice': ['gini', 'entropy']
    },
    {
        'name': 'min_samples_split',
        'type': 'int',
        'min': 2,
        'max': 4,
        'choice': []
    }    
]


target_metrics = 'accuracy'
options = {
    'num_samples': 10,
    'stopping_criteria': {
        'training_iteration': 3,
    },
    'max_concurrent': 16
}

If it is a choice, then we need to use other model 

In [112]:
analysis = tuner.run(SklearnRFClassifier, parameters, target_metrics, options, search_algorithm="BOHB")

Trial name,status,loc,criterion,max_depth,min_samples_split,n_estimators,acc,iter,total time (s)
H1stModelTrainable_7a51cdc4,TERMINATED,,gini,4,3,50,0.809091,3,2.82189
H1stModelTrainable_7a51faec,TERMINATED,,gini,3,2,100,0.805859,3,4.30478
H1stModelTrainable_7a52265c,TERMINATED,,entropy,7,3,150,0.81798,3,15.758
H1stModelTrainable_7a524f24,TERMINATED,,gini,6,4,100,0.817374,3,7.48806
H1stModelTrainable_7a527cba,TERMINATED,,entropy,5,3,150,0.81202,3,10.7694
H1stModelTrainable_7a52a71c,TERMINATED,,entropy,7,4,100,0.818182,3,10.3748
H1stModelTrainable_7a52ced6,TERMINATED,,gini,6,2,50,0.815657,3,4.13334
H1stModelTrainable_7a52f5aa,TERMINATED,,gini,8,2,100,0.818889,3,11.2498
H1stModelTrainable_7a5322a0,TERMINATED,,gini,8,2,100,0.818687,3,11.0547
H1stModelTrainable_7a534a5a,TERMINATED,,entropy,3,2,100,0.804949,3,4.95403


took   41.92 seconds (  0 hours 0.70 minutes)


In [113]:
analysis.head(20)

,model_version,mean_accuracy,training_iteration,config/n_estimators,config/max_depth,config/criterion,config/min_samples_split
7,n_est=100|max_d=8|crite=gini|min_s=2|55623042,0.819495,1,100,8,gini,2
2,n_est=150|max_d=7|crite=entropy|min_s=3|55293125,0.819192,1,150,7,entropy,3
5,n_est=100|max_d=7|crite=entropy|min_s=4|10391647,0.819192,2,100,7,entropy,4
8,n_est=100|max_d=8|crite=gini|min_s=2|55675652,0.819091,1,100,8,gini,2
3,n_est=100|max_d=6|crite=gini|min_s=4|54972321,0.818081,1,100,6,gini,4
6,n_est=50|max_d=6|crite=gini|min_s=2|15841532,0.815657,3,50,6,gini,2
4,n_est=150|max_d=5|crite=entropy|min_s=3|14609608,0.812020,3,150,5,entropy,3
0,n_est=50|max_d=4|crite=gini|min_s=3|15031358,0.809091,3,50,4,gini,3
1,n_est=100|max_d=3|crite=gini|min_s=2|14782684,0.806970,2,100,3,gini,2
9,n_est=100|max_d=3|crite=entropy|min_s=2|14688548,0.805455,2,100,3,entropy,2


In [107]:
print(analysis.shape)
print(analysis.model_version.nunique())

(10, 7)
10


In [50]:
# cols = ['mean_accuracy', 'config/lr', 'config/n_layer', 'config/units']
cols = ['model_version', 'mean_accuracy', 'training_iteration'] + [f'config/{param["name"]}' for param in parameters]
analysis.dataframe()[cols].sort_values('mean_accuracy', ascending=False).head(20)

,model_version,mean_accuracy,training_iteration,config/n_estimators,config/max_depth,config/criterion,config/min_samples_split
12,3_n_estimators=50_max_depth=8_criterion=gini_m...,0.820505,3,50,8,gini,2
3,1_n_estimators=100_max_depth=8_criterion=entro...,0.820000,1,100,8,entropy,4
7,2_n_estimators=50_max_depth=8_criterion=gini_m...,0.819798,2,50,8,gini,4
4,2_n_estimators=50_max_depth=6_criterion=entrop...,0.819596,2,50,6,entropy,3
0,3_n_estimators=50_max_depth=8_criterion=gini_m...,0.818990,3,50,8,gini,2
14,3_n_estimators=100_max_depth=6_criterion=gini_...,0.818990,3,100,6,gini,3
10,3_n_estimators=150_max_depth=7_criterion=gini_...,0.818990,3,150,7,gini,2
2,1_n_estimators=100_max_depth=6_criterion=gini_...,0.818990,1,100,6,gini,4
19,2_n_estimators=150_max_depth=8_criterion=gini_...,0.818889,2,150,8,gini,3
1,2_n_estimators=150_max_depth=8_criterion=entro...,0.818485,2,150,8,entropy,2


In [21]:
analysis.dataframe()[cols].sort_values('mean_accuracy', ascending=False).head(20).mean_accuracy.mean()

0.820409090909091

In [67]:
from datetime import datetime

In [100]:
datetime.now().strftime("%H%M%S%f")

'163009.3f'

In [80]:
datetime.now().strftime('HH:MM:SS.ffffff')

'HH:MM:SS.ffffff'

In [71]:
import random

In [74]:
random.randint(0,100000)

33717

In [70]:
datetime.time()

TypeError: descriptor 'time' of 'datetime.datetime' object needs an argument

In [ ]:
a = int(a.strftime('%Y%m%d'))